<a href="https://colab.research.google.com/github/yoon777/cv_project/blob/main/text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (739 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123622 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!apt-get update
!apt-get install tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.8 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://

In [ ]:
pip install pdf2image

In [ ]:
pip install pdfplumber pymupdf opencv-python pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.6 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import fitz  # PyMuPDF
import cv2
import pytesseract
from PIL import Image
import numpy as np

# 1. 텍스트 추출
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                full_text += text + "\n"
    return full_text

# 2. 표 추출 (텍스트 기반)
def extract_tables_from_pdf(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_tables = page.extract_tables()
            for table in page_tables:
                tables.append(table)
    return tables

In [ ]:
import os
import pdfplumber

# 필요한 라이브러리들 (OCR에 필요한 라이브러리 추가)
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
from PIL import Image

# 경로 설정
pdf_path = '2024 용인 정책.pdf'
output_folder = "output"

# 폴더 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# PDF 열기
with pdfplumber.open(pdf_path) as pdf:
    num_pages = len(pdf.pages)

    # 각 페이지별로 텍스트, 표, 그래프를 추출
    for i in range(num_pages):
        page = pdf.pages[i]

        # 페이지 번호 정보
        page_number = i + 1
        print(f"Processing page {page_number}...")

        # 텍스트 추출
        print(f"Extracting text from page {page_number}...")
        page_text = page.extract_text()

        # 텍스트 후처리: 20자 이하인 줄은 앞 줄과 합치기
        if page_text:
            lines = page_text.split('\n')
            processed_lines = []
            for j in range(len(lines)):
                if j > 0 and len(lines[j]) <= 20:  # 현재 줄이 20자 이하인 경우
                    processed_lines[-1] += " " + lines[j]  # 이전 줄과 합침
                else:
                    processed_lines.append(lines[j])  # 그렇지 않으면 그대로 추가
            processed_text = '\n'.join(processed_lines)
        else:
            processed_text = ""

        # 텍스트 저장
        text_output_path = os.path.join(output_folder, f"page_{page_number}_text.txt")
        with open(text_output_path, 'w', encoding='utf-8') as text_file:
            text_file.write(processed_text)

        # 표 추출
        print(f"Extracting tables from page {page_number}...")
        tables = page.extract_tables()
        if tables:
            for idx, table in enumerate(tables):
                df = pd.DataFrame(table)
                table_output_path = os.path.join(output_folder, f"page_{page_number}_table_{idx + 1}.csv")
                df.to_csv(table_output_path, index=False, encoding='utf-8-sig')

print("Extraction completed.")

Processing page 1...
Extracting text from page 1...
Extracting tables from page 1...
Processing page 2...
Extracting text from page 2...
Extracting tables from page 2...
Processing page 3...
Extracting text from page 3...
Extracting tables from page 3...
Processing page 4...
Extracting text from page 4...
Extracting tables from page 4...
Processing page 5...
Extracting text from page 5...
Extracting tables from page 5...
Processing page 6...
Extracting text from page 6...
Extracting tables from page 6...
Processing page 7...
Extracting text from page 7...
Extracting tables from page 7...
Processing page 8...
Extracting text from page 8...
Extracting tables from page 8...
Processing page 9...
Extracting text from page 9...
Extracting tables from page 9...
Processing page 10...
Extracting text from page 10...
Extracting tables from page 10...
Processing page 11...
Extracting text from page 11...
Extracting tables from page 11...
Processing page 12...
Extracting text from page 12...
Extra

In [ ]:
import os
import pandas as pd

# 경로 설정
output_folder = "output"
final_output_folder = "final_output"

# 폴더 생성
if not os.path.exists(final_output_folder):
    os.makedirs(final_output_folder)

# 출력 폴더 내의 파일 목록 가져오기
files = os.listdir(output_folder)

# 페이지 번호별로 텍스트 및 표 파일 결합
pages = set(f.split('_')[1] for f in files if f.startswith('page_'))

for page_number in pages:
    # 텍스트 파일 읽기
    text_file_path = os.path.join(output_folder, f"page_{page_number}_text.txt")
    if os.path.exists(text_file_path):
        with open(text_file_path, 'r', encoding='utf-8') as text_file:
            lines = text_file.readlines()
            # 텍스트 데이터를 DataFrame으로 변환 (한 줄이 한 행으로 들어가도록)
            text_df = pd.DataFrame({'Content': [line.strip() for line in lines]})
    else:
        text_df = pd.DataFrame({'Content': []})

    # 해당 페이지의 모든 표 파일 읽기
    table_dfs = []
    idx = 1
    while True:
        table_file_path = os.path.join(output_folder, f"page_{page_number}_table_{idx}.csv")
        if os.path.exists(table_file_path):
            table_df = pd.read_csv(table_file_path, encoding='utf-8-sig')
            table_dfs.append(table_df)
            idx += 1
        else:
            break

    # 텍스트와 표를 하나의 DataFrame으로 결합 (텍스트 뒤에 표를 바로 이어서 결합)
    combined_dfs = [text_df]
    for table_idx, table_df in enumerate(table_dfs, start=1):
        # 구분자 추가
        separator_df = pd.DataFrame({"Content": [f"<{page_number} 페이지의 표 {table_idx} 내용>"]})
        combined_dfs.append(separator_df)
        combined_dfs.append(table_df)

    combined_df = pd.concat(combined_dfs, ignore_index=True)

    # 페이지별 결합된 CSV 저장
    combined_output_path = os.path.join(final_output_folder, f"page_{page_number}_combined.csv")
    combined_df.to_csv(combined_output_path, index=False, encoding='utf-8-sig')

print("Merging completed.")

Merging completed.


In [ ]:
# 페이지별 결합된 CSV 파일 합치기
print("Combining all page CSV files into one...")
all_combined_df = pd.DataFrame()
for page_number in pages:
    file_path = os.path.join(final_output_folder, f"page_{page_number}_combined.csv")
    if os.path.exists(file_path):
        page_df = pd.read_csv(file_path, encoding='utf-8-sig')
        all_combined_df = pd.concat([all_combined_df, page_df], ignore_index=True)

# 왼쪽에 빈 셀이 있으면 오른쪽 셀의 내용을 왼쪽으로 이동
all_combined_df['Content'] = all_combined_df['Content'].fillna(method='ffill')

# 최종 결합된 CSV 저장
all_combined_df.to_csv('final_combined.csv', index=False, encoding='utf-8-sig')

print("All pages combined and saved to final_output/final_combined.csv.")

Combining all page CSV files into one...
All pages combined and saved to final_output/final_combined.csv.


<ipython-input-13-f84827f5a504>:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_combined_df['Content'] = all_combined_df['Content'].fillna(method='ffill')


In [ ]:
import pandas as pd

# 최종 결합된 CSV 파일 가져오기
final_combined_csv_path = 'final_combined.csv'
all_combined_df = pd.read_csv(final_combined_csv_path, encoding='utf-8-sig')

# 왼쪽에 빈 셀이 있으면 오른쪽 셀의 내용을 왼쪽으로 이동
all_combined_df = all_combined_df.apply(lambda row: row.bfill(), axis=1).iloc[:, 0].to_frame(name='Content')

# 최종 정리된 CSV 저장
final_processed_csv_path = 'final_combined_processed.csv'
all_combined_df.to_csv(final_processed_csv_path, index=False, encoding='utf-8-sig')

print(f"Processed CSV saved to {final_processed_csv_path}.")

Processed CSV saved to final_combined_processed.csv.


<ipython-input-42-1dba56e8c428>:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_combined_df = all_combined_df.apply(lambda row: row.bfill(), axis=1).iloc[:, 0].to_frame(name='Content')


In [ ]:
import os
import pdfplumber
import pandas as pd
import fitz  # PyMuPDF
import cv2
import pytesseract
from PIL import Image
import numpy as np

# 경로 설정
output_folder = "output"
final_output_folder = "final_output"

# 폴더 생성
if not os.path.exists(final_output_folder):
    os.makedirs(final_output_folder)

# 출력 폴더 내의 파일 목록 가져오기
files = os.listdir(output_folder)

# 페이지 번호별로 텍스트 및 표 파일 결합
pages = set(f.split('_')[1] for f in files if f.startswith('page_'))

for page_number in pages:
    # 텍스트 파일 읽기
    text_file_path = os.path.join(output_folder, f"page_{page_number}_text.txt")
    if os.path.exists(text_file_path):
        with open(text_file_path, 'r', encoding='utf-8') as text_file:
            lines = text_file.readlines()
            # 텍스트 데이터를 DataFrame으로 변환 (한 줄이 한 행으로 들어가도록)
            text_df = pd.DataFrame({'Content': [line.strip() for line in lines]})
    else:
        text_df = pd.DataFrame({'Content': []})

    # 해당 페이지의 모든 표 파일 읽기
    table_dfs = []
    idx = 1
    while True:
        table_file_path = os.path.join(output_folder, f"page_{page_number}_table_{idx}.csv")
        if os.path.exists(table_file_path):
            table_df = pd.read_csv(table_file_path, encoding='utf-8-sig')
            table_dfs.append(table_df)
            idx += 1
        else:
            break

    # 텍스트와 표를 하나의 DataFrame으로 결합 (텍스트 뒤에 표를 바로 이어서 결합)
    combined_dfs = [text_df]
    for table_idx, table_df in enumerate(table_dfs, start=1):
        # 구분자 추가
        separator_df = pd.DataFrame({"Content": [f"<{page_number} 페이지의 표 {table_idx} 내용>"]})
        combined_dfs.append(separator_df)
        combined_dfs.append(table_df)

    combined_df = pd.concat(combined_dfs, ignore_index=True)

    # 페이지별 결합된 CSV 저장
    combined_output_path = os.path.join(final_output_folder, f"page_{page_number}_combined.csv")
    combined_df.to_csv(combined_output_path, index=False, encoding='utf-8-sig')

print("Merging completed.")

# 페이지별 결합된 CSV 파일 합치기
print("Combining all page CSV files into one...")
all_combined_df = pd.DataFrame()
for file in sorted(os.listdir(final_output_folder)):
    if file.endswith("_combined.csv"):
        file_path = os.path.join(final_output_folder, file)
        page_df = pd.read_csv(file_path, encoding='utf-8-sig')
        all_combined_df = pd.concat([all_combined_df, page_df], ignore_index=True)

# 왼쪽에 빈 셀이 있으면 오른쪽 셀의 내용을 왼쪽으로 이동
all_combined_df['Content'] = all_combined_df['Content'].fillna('').apply(lambda row: ' '.join([str(cell) for cell in row if str(cell).strip() != '']))

# 최종 결합된 CSV 저장
all_combined_df.to_csv('final_combined.csv', index=False, encoding='utf-8-sig')

print("All pages combined and saved to final_output/final_combined.csv.")

FileNotFoundError: [Errno 2] No such file or directory: 'output_2'